In [ ]:
# Use this cell to import all the required packages and methods
import typing
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import PIL
import tensorflow as tf
from functools import reduce
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import load_img, to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
sns.set_style('whitegrid')

In [ ]:
# Use this cell to define a function that loads images and their labels

# Define a function that loads images and their labels
def load_data(mainfolder: str = "weather"):
    '''
    Loads the images from the main folder in a list and creates a list
    containing labels for the images

    Args:
        mainfolder (str): A string that describes the address of the parent data
            folder in the memory
    
    Returns:
        list_of_images (PIL): A list containing PIL Image instances for each image
        image_labels (List[str]): A list containing their respective labels in the form 
            of a string
    '''
    list_of_images = []
    image_labels: typing.List[str] = []
    
    for dir in os.listdir(mainfolder):
        for imgs in os.listdir(mainfolder+'/'+dir):
            list_of_images.append(load_img(mainfolder+"/"+dir+'/'+imgs))
            image_labels.append(dir)
    
    return list_of_images, image_labels

Finally, use the *load_data()* function to load the training and testing data.

In [ ]:
# Use this cell to load the training and testing data sets and store them in the appropriate variables

# Load the training data
X_train, y_train = load_data('weather/train')

# Load the testing data
X_test, y_test = load_data('weather/test')

In [ ]:
for img, label in zip(X_train, y_train):
    print(img, label)

## Task 3 - Explore the data

In [ ]:
train_dataset = pd.DataFrame(zip(X_train, y_train), columns=['image', 'label'])
test_dataset = pd.DataFrame(zip(X_test, y_test), columns=['image', 'label'])

In [ ]:
train_sample_each_category = train_dataset.groupby(by=['label']).sample(1)
test_sample_each_category = test_dataset.groupby(by=['label']).sample(1)

In [ ]:
!explorer .

In [ ]:
# Use this cell to view a few images from the testing data set for each class
fig, ax = plt.subplots(1, 3, figsize=(20, 20), layout='constrained')
count = 0
for img, label in zip(test_sample_each_category['image'], test_sample_each_category['label']):
    ax[count].set_title(label, fontsize='24')
    ax[count].imshow(img)
    ax[count].set_xlabel(f"Size of Image is: {img.size}", fontsize=18)
    ax[count].grid(alpha=0)
    count += 1
fig.savefig('classwise_images.png')

Print the number of training and testing data points available to you.

In [ ]:
# Use this cell to inspect the number of training and testing data points available

print(f"Length of trainig data: {train_dataset['image'].count()}")
print(f"Length of test data: {test_dataset['image'].count()}")

In [ ]:
ax = sns.barplot(x=['Training', 'Testing'], y=[train_dataset['image'].count(), test_dataset['image'].count()])
ax.set_title("Length of datasets")
ax.bar_label(ax.containers[0])
plt.savefig('dataset_len.png')

Store the number of classes in this classification exercise.

In [ ]:
# Use this cell to extract and store the total class count

# Extract and store the total class count
n_classes =  np.unique(y_test).shape[0]
print('Number of classes =', n_classes)

visualze the class balance in the training data set.

In [ ]:
# Use this cell to visualize the class balance in the training data set
ax = sns.barplot(train_dataset.groupby(by='label').count().reset_index(), x='label', y='image')
ax.set_title("Class Balance in training Dataset")
ax.bar_label(ax.containers[0], fontsize=10)
plt.savefig("class_balance.png")

In [ ]:
training_image_sizes_df = pd.DataFrame()
testing_image_sizes_df = pd.DataFrame()

In [ ]:
training_image_sizes_df['width'], training_image_sizes_df['height'], training_image_sizes_df['aspect_ratio']  = zip(*train_dataset['image'].apply(lambda x: (x.size[0], x.size[1], round(x.size[0] / x.size[1], 2))))
testing_image_sizes_df['width'], testing_image_sizes_df['height'], testing_image_sizes_df['aspect_ratio'] = zip(*test_dataset['image'].apply(lambda x: (x.size[0], x.size[1], round(x.size[0] / x.size[1], 2))))

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
points = ax[0].scatter(training_image_sizes_df['width'], training_image_sizes_df['height'], alpha=0.5, s=training_image_sizes_df['aspect_ratio']*100, picker=True)
ax[0].set_title("Image Resolution")
ax[0].set_xlabel("Width", size=14)
ax[0].set_ylabel("Height", size=14)

points = ax[1].scatter(training_image_sizes_df['width'], training_image_sizes_df['height'], alpha=0.5, s=training_image_sizes_df['aspect_ratio']*100, picker=True)
ax[1].set_title("Zoomed to 1400 x 1400 resolution")
ax[1].set_xlabel("Width", size=14)
ax[1].set_ylabel("Height", size=14)
ax[1].set_ylim(0, 800)
ax[1].set_xlim(0, 1400)

points = ax[2].scatter(training_image_sizes_df['width'], training_image_sizes_df['height'], alpha=0.5, s=training_image_sizes_df['aspect_ratio']*100, picker=True)
ax[2].set_title("Zoomed to 250 x 500 resolution")
ax[2].set_xlabel("Width", size=14)
ax[2].set_ylabel("Height", size=14)
ax[2].set_ylim(100, 250)
ax[2].set_xlim(0, 500)

plt.savefig("image_dimensions.png")

# Stage 2 - Data Preparation
In this stage, you will perform some basic essential data preparation methods on your image data so that they are ready for use in CNNs. 

To prepare the data, you will first need to resize the images in the training and testing data sets to the same size. This can be done using some methods provided to us in the *tensorflow* library. Second, you will convert the data into a format that is suitable to be fed into a Keras CNN model.

You will achieve this by completing the following tasks:
- Task 4 - Resize all images to the same dimension
- Task 5 - Prepare the data for feeding into CNN

## Task 4 - Resize all images to the same dimensions


In [ ]:
# Use this cell to define a function that resizes image dimensions

# Define a function that resizes image dimensions
def resize_images(input_images: typing.List[PIL] | np.array = [], new_dims: typing.Tuple[int, int] = 200, col_name: str = None):
    '''
    Resizes all the images in a list to a square with its side given.

    Args:
        input_images: A list of images to be resized. Each input image 
            must be a PIL image.
        new_dims: An integer specifying the desired dimensions of the output 
            images. Each output image will have the same height and width.
        col_name (str): Column name which consists the images in dataset.
            (use when pass an dataframe as input_images)

    Returns:
        resized_images: A list of resized images. Each output image is a PIL 
            image with the specified dimensions.
    '''

    # Type checker for the input images.

    resized_images = [array_to_img(tf.image.resize(img_to_array(x), new_dims)) for x in input_images]
    return resized_images

Next, decide on a new image dimension.


In [ ]:
# Use this cell to set the new dimensions for all images

# Set the new dimensions for all images
new_image_dims = (64, 64)

Use the *resize_images()* method to resize your training and testing images.

In [ ]:
# Use this cell to standardize the dimensions for all images in the training and testing data sets

# Standardize the dimensions for all images in the training data set
X_train = np.array(resize_images(X_train, new_image_dims))

# Standardize the dimensions for all images in the testing data set
X_test = np.array(resize_images(X_test, new_image_dims))

In [ ]:
X_train.shape

View some images from the resized data set.

In [ ]:
train_dataset = pd.DataFrame(zip(X_train, y_train), columns=['image', 'label'])
test_dataset = pd.DataFrame(zip(X_test, y_test), columns=['image', 'label'])

In [ ]:
# Use this cell to view a few images from the testing data set for each class
test_sample_each_category = test_dataset.groupby(by=['label']).sample(1)
fig, ax = plt.subplots(1, 3, figsize=(20, 20), layout='constrained')
fig.text(0.43, 0.7, "Test Dataset Images", fontsize=26)
count = 0
for img, label in zip(test_sample_each_category['image'], test_sample_each_category['label']):
    ax[count].set_title(label, fontsize='24')
    ax[count].imshow(img)
    ax[count].set_xlabel(f"Size of Image is: {img.size}", fontsize=18)
    ax[count].grid(alpha=0)
    count += 1
plt.show()

In [ ]:
train_sample_each_category = train_dataset.groupby(by=['label']).sample(1)
fig, ax = plt.subplots(1, 3, figsize=(20, 20), layout='constrained')
# fig.text(0.43, 0.7, "Training Dataset Images", fontsize=26)
count = 0
for img, label in zip(train_sample_each_category['image'], train_sample_each_category['label']):
    ax[count].set_title(label, fontsize='24')
    ax[count].imshow(img)
    ax[count].set_xlabel(f"Size of Image is: {img.size}", fontsize=18)
    ax[count].grid(alpha=0)
    count += 1
plt.savefig('resized_imgs.png')

Finally, save the resized data set into variables that will be useful later.

In [ ]:
# Use this cell to store the resized images and labels for later use

# Store the resized images for later use
X_pre_NN = X_train

# Store the labels for later use
y_pre_NN = y_train

In [ ]:
pre_NN_df = pd.DataFrame(zip(X_pre_NN, y_pre_NN), columns=['image', 'label'])

## Task 5 - Prepare the data for feeding into CNN


### Description

In this section, you will perform a set of data preparation steps that enables CNNs to work on the data. Currently, *X_train* and *X_test* are lists of images, whereas *y_train* and *y_test* are lists of strings. Keras models can accept data in the form of *numpy* arrays. Therefore, you will convert the data into arrays.

For the input data, you need to convert the list of images into an *numpy* array. Note that the pixel gray levels are currently in the range $[0, 255]$. You will rescale these to $[0, 1]$. This is a common pre-processing step in image classification tasks, as it helps the model to converge faster during training.

For the output data, you will need to encode the different labels as one-hot encoded vectors.

In [ ]:
X_train = np.array([img_to_array(x) / 255 for x in X_train])

In [ ]:
X_train.shape

In [ ]:
X_train[0]

In [ ]:
X_test = np.array([img_to_array(x) / 255 for x in X_test])

In [ ]:
X_test.shape

Now convert the image labels to one-hot encoded vectors.

In [ ]:
num_classes = len(set(y_train))

In [ ]:
labels = list(set(y_train))

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
y_train_encoded = label_encoder.fit_transform(y_train)

In [ ]:
y_train = to_categorical(y_train_encoded, num_classes=num_classes)

In [ ]:
classes = ['Cloudy', 'Sunny', 'Rainy']
encoded_classes = [list(y) for y in set([tuple(x) for x in y_train])]

In [ ]:
{k:v for k, v in zip(classes, encoded_classes)}

In [ ]:
y_train.tolist()


In [ ]:
y_test_encoded = label_encoder.fit_transform(y_test)

In [ ]:
y_test = to_categorical(y_test_encoded, num_classes=num_classes)

In [ ]:
y_test

In [ ]:
array_to_img(X_train[0])

In [ ]:
y_train[0]

Finally, save the input dimensions for the CNNs in a variable so that you can call or refer to it when building CNN models.

In [ ]:
X_train.shape[1], X_train.shape[2], X_train.shape[3]

In [ ]:
# Use this cell to store the input dimensions for the CNNs

# Store the input dimensions for the CNNs
inputdims = X_train.shape[1], X_train.shape[2], X_train.shape[3]

In [ ]:
inputdims

# Stage 3 - Simple Model
In this stage, you will build, train and evaluate a basic CNN model on the data and analyze its performance. You will build the model using a function. This will help you change the configuration of your model during execution. You will also test the performance of the performance of the model by training it multiple times and then judging the distribution of final accuracies and loss values.

You will do all this with the help of the following tasks:
- Task 6 - Define a function to build a CNN model
- Task 7 - Create a simple CNN model and analyze its performance

## Task 6 - Define a function to build a CNN model


Define the *create_cnn()* function.

In [ ]:
# Use this cell to define a function that creates and compiles a CNN

# Define a function that creates and compiles a CNN
def create_cnn(layers_config: typing.List[str], input_dims: typing.Tuple[int],  num_classes: int, learning_rate_value: float = 0.01):
    '''
    Creates and compiles a convolutional neural network (CNN) with the specified layers configuration and learning rate.

    Args:
        layers_config (List[str]): A list of strings specifying the configuration of each layer in the CNN. 
        input_dims (tuple): Tuple specifying the dimensions of inputs for the models to handle.
        num_classes: Number of classes to classify in the output.
        learning_rate_value (float): A float specifying the learning rate to be used by the optimizer during training.
    
    Returns:
        cnn: A compiled CNN with the specified layers configuration and learning rate.
    '''

    # Initializing Sequential model.
    cnn = Sequential()

    # Add Input layer with the provided input dimensions.
    cnn.add(Input(shape=input_dims))

    # Iterate over layers_config to add layers to the model.
    for layer_str in layers_config:
        # Extract layer type and parameters.
        layer_type, *params = layer_str.split('_')

        if layer_type == 'c': # Conv2D layer.
            filters, kernel_size = map(int, params)
            cnn.add(Conv2D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same'))
        elif layer_type == 'm': # MaxPooling2D layer.
            pool_size = int(params[0])
            cnn.add(MaxPooling2D(pool_size=(pool_size, pool_size)))
        elif layer_type == 'f': # flatten Layer.
            cnn.add(Flatten())
        elif layer_type == 'd': # dense layer.
            units = int(params[0])
            cnn.add(Dense(units=units, activation='relu'))

    cnn.add(Dense(num_classes, activation='softmax'))

    cnn.compile(optimizer=Adam(learning_rate=learning_rate_value),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

    return cnn

### Checklist
- Defined a function *create_cnn()* with given keyword arguments and return variables
- Followed the recommended specifications in the model

## Task 7 - Create a simple CNN model and analyze its performance

### Description

In this task, you will create a simple CNN model and train it on the training data multiple times and record its performance in each training iteration. You will then analyze the model's performance by summarizing its performance over the various training trials.

The reason why we want to train it multiple times is that your model may not train the same way each time. To get a good understanding of a model's performance, you need to train it a few times under the same settings and conditions and record its performance each time. You can then judge the model based on the distribution of its performances.

First, save the number of trials in a variable.

In [ ]:
# Use this cell to set the number of trials for each model training instance

# Set the number of trials for each model training instance
num_trials = 10

Now save the number of epochs and the validation split in a variable.

In [ ]:
# Use this cell to set the number of CNN training epochs and the validation split fraction

# Set the number of epochs for CNN training
n_epochs = 10

# Set the validation split fraction
val_split = 0.2

Finally, create train and evaluate your CNN model.

In [ ]:
inputdims

In [ ]:
num_classes

In [ ]:
# Use this cell to create, train and evaluate your simple CNN model on the data multiple times and store and view the performance results
training_hist = [None] * num_trials
performance_df = pd.DataFrame()

layers_config = ['c_2_3', 'm_2', 'c_4_3', 'm_2', 'f', 'd_8']
learning_rate_value = 0.01

for i in range(0, num_trials):
    results = []
    cnn = create_cnn(layers_config=layers_config, input_dims=inputdims, num_classes=num_classes, learning_rate_value=learning_rate_value)

    print(f"Training iteration {i}")
    cnn.summary()
    print('\n')
    cnn_history = cnn.fit(X_train, y_train, validation_split=val_split, epochs=n_epochs)

    training_hist[i] = pd.DataFrame(cnn_history.history)
    training_hist[i]['epoch'] = cnn_history.epoch

performance_df = reduce(lambda left, right: pd.merge(left, right, how='outer'), training_hist)
performance_df.loc['Mean'] = performance_df.mean()
performance_df.loc['Median'] = performance_df.median()
performance_df.loc['Max'] = performance_df.max() 

In [ ]:
plt.figure(figsize = (14, 4))

sns.lineplot(data = performance_df, x = 'epoch', y = 'accuracy', color = 'red', label = 'Train')
sns.lineplot(data = performance_df, x = 'epoch', y = 'val_accuracy', color = 'blue', label = 'Validation')
plt.xlabel('Iterations')
plt.ylabel('Accuracy')
plt.title('Accuracy per Iteration');
plt.savefig("simple_model_result.png")

### Checklist
- Saved values for number of trials, number of epochs, and validation split in variables
- Created and trained a simple CNN model multiple times and evaluated its performance
- Displayed the model's performance over multiple trials and the summary statistics in the given format

# Stage 4 - Data Augmentation
Data augmentation is a technique that is commonly used in deep learning to artificially increase the size of the training data set.

You can add augmented images to your your training data to improve the class balance in the training data set and also to increase the size of the training data.

In this stage, you will augment your training data to increase the training data size and improve the class balance in the training data set. You will then retrain your basic CNN model on the augmented data set and analyze its performance over multiple training trials.

In the first three tasks of this stage, you will write functions and helper functions to perform data augmentation. Whereas in the fourth task, you will use those functions to actually perform data augmentation. The tasks are given below:
- Task 8 - Create a transformed image
- Task 9 - Divide the data according to class
- Task 10 - Augment the data
- Task 11 - Create a simple CNN model using the augmented data and analyze its performance

## Task 8 - Create a transformed image

### Description
In the case of image classification, data augmentation is done by applying random transformations to the images in the training data set, such as rotation, flipping, and zooming. By doing so, the model is exposed to a greater variety of training samples, which can help improve its ability to generalize to unseen data.

In this task, you will define a function to randomly transform a given image using one of the predefined transformations. This function will be used later to augment the data. 

But recall that we performed some data preparation steps on our data set in Task 5. Recall also that we saved our data in two variables *X_pre_NN* and *y_pre_NN*. We will perform data augmentation using this unprocessed data.

Retrieve the training data that you saved in an earlier stage before conducting data preparation on it for feeding into CNNs.

In [ ]:
# Use this cell to retrieve the training data that you saved in an earlier stage

# Retrieve the training images
X_train = X_pre_NN

# Retrieve the training labels
y_train = y_pre_NN

Now, define the function *random_transform()* that takes in an image and creates a new image from it using a random transformation.

In [ ]:
# Use this cell to define a function that takes in an input image and creates a new image out of it using some random augmentation

# Define a function that takes in an input image and creates a new image out of it using some random augmentation
def random_transform(input_image):
    """
    Takes in an input image and creates a new image out of it using a  
        random transformation.
    Args:
        input_image: A PIL image object.

    Returns:
        output_image: A PIL image object, the augmented image.
    """

    # convert PIL image to numpy array.
    input_array = img_to_array(input_image)

    # Randomly choose transformation.
    transformation = np.random.choice([
        'flip_left_right',
        'flip_up_down',
        'rot90'
    ])

    if transformation == 'flip_left_right':
        print("Flipping the image left to right.")
        output_array = tf.image.flip_left_right(input_array)
    elif transformation == 'flip_up_down':
        print("Flipping the image up to down.")
        output_array = tf.image.flip_up_down(input_array)
    elif transformation == 'rot90':
        k = np.random.randint(1, 4)  # Randomly choose rotation angle (1, 2, or 3)
        print("Rotating the image")
        output_array = tf.image.rot90(input_array, k)

    # convert numpy array back to PIL Image.
    output_image = array_to_img(output_array)

    return output_image

You can now test your function on any input image and view the results. Note that if you defined your function correctly, each time you run the following cell, a random augmentation would be performed.

In [ ]:
X_train[0]

In [ ]:
# Use this cell to apply the "random_transform()" function on an image and view the results
# Note: This is a sample execution and the actual augmentation function will be defined in Task 10

random_transform(X_train[0])

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(10, 10), layout='constrained')
# fig.text(0.43, 0.7, "Training Dataset Images", fontsize=26)
count = 0
for img in [random_transform(X_train[0]), random_transform(X_train[1]), random_transform(X_train[2])]:
    ax[count].imshow(img)
    ax[count].set_xlabel(f"Size of Image is: {img.size}", fontsize=18)
    ax[count].grid(alpha=0)
    count += 1
plt.savefig('transformed_imgs.png')

### Checklist
- Retrieved data stored in *X_pre_NN* and *y_pre_NN*
- Defined the *random_transform()* function using the keyword arguments and return variables described above
- Experimented with the *random_transform()* function and verified that the function randomly augments an input image

## Task 9 - Divide the data by class


Visualize the class balance in the training data set using a bar plot or a count plot.

In [ ]:
train_df = pd.DataFrame(zip(X_train, y_train), columns=['image', 'label']).groupby(by='label').count()

In [ ]:
# Use this cell to visualize the class balance in the training data set
ax = sns.barplot(train_df.reset_index(), x='label', y='image')
ax.bar_label(ax.containers[0])
plt.show()

You can see that the classes in the training data are quite imbalanced. Moreover, the number of training samples is quite small as well.

Now, define the function *divide_data_by_class()* that divides the input images and labels into subsets based on their corresponding class labels.

In [ ]:
# Use this cell to define a function that takes in the list of training images and labels and returns them class-wise

# Define a function that takes in the list of training images and returns them class-wise
def divide_data_by_class(input_images, image_labels):
    '''
    Divides the input images and labels into subsets based on their corresponding class labels.

    Args:
        input_images: A list of input images to be divided based on class 
            labels. Each input image must be a PIL image.
        image_labels: A list that contains corresponding labels for each 
            image in input_images. Each label is a string.

    Returns:
        classwise_images: A list of lists, where each sublist contains the 
            input images corresponding to a unique class label.
        classwise_labels: A list of lists, where each sublist contains the 
            labels corresponding to the input images in classwise_images list.
    '''

   # Initialize dictionaries to store images and labels by class
    classwise_images = {}
    classwise_labels = {}

    # Iterate over input images and labels
    for image, label in zip(input_images, image_labels):
        # Check if class label already exists in dictionaries
        if label not in classwise_images:
            classwise_images[label] = []
            classwise_labels[label] = []
        
        # Add image and label to respective class
        classwise_images[label].append(image)
        classwise_labels[label].append(label)

    # Convert dictionaries to lists of lists
    classwise_images = list(classwise_images.values())
    classwise_labels = list(classwise_labels.values())

    return classwise_images, classwise_labels

You can now use the *divide_data_by_class()* function on the data set and view the results.

### Checklist
- Visualized class balance in the data set
- The *divide_data_by_class()* function returns two lists that contain three lists each
- Experimented with the *divide_data_by_class()* function and ensured that it is functioning properly

## Task 10 - Augment the training data

### Description

With these two functions, *random_transform()* and *divide_data_by_class()* in hand, you can define the main augmentation function called *augment_data()*. This function takes in an image data set and the factor by which the data set needs to be augmented. This function should:
1. Divide the data set into different classes using the *divide_data_by_class()* function.
2. Calculate the final size of each class. This is done by multiplying the factor by the size of the largest class.
3. Use the *random_transform()* function to create images till all three classes achieve the required size.

After defining the function, try augmenting the data set and visualize the class balance again to verify your results

Define the *augment_data()* function that works on the original training data and produces an augmented training data set from it.

In [ ]:
# Use this cell to define a function that augments the training data

# Define a function to augment the training data
def augment_data(input_images, image_labels, data_size_factor):
    '''
    Augments the training data by randomly applying data augmentation techniques.

    Args:
        input_images: A list of input images.
        image_labels: A list of labels for the input images.
        data_size_factor: A scaling factor for the size of the augmented data.
            This will be used to calculate the final size of each class by 
            multiplying data_size_factor with the size of the largest class and 
            then rounding to the nearest integer.

    Returns:
        new_images: The augmented images
        new_labels: The labels corresponding to the new images
    '''

    # Divide input data by class
    classwise_images, classwise_labels = divide_data_by_class(input_images, image_labels)

    # Find the size of the largest class
    max_class_size = max(len(images) for images in classwise_images)

    # Calculate target size for each class after augmentation
    target_size = round(max_class_size * data_size_factor)

    # Augment data for each class
    new_images = []
    new_labels = []
    for images, labels in zip(classwise_images, classwise_labels):
        num_images_to_augment = target_size - len(images)
        if num_images_to_augment > 0:
            new_images.extend(images)
            new_labels.extend(labels)
            for _ in range(num_images_to_augment):
                # Randomly choose an image to augment
                image_to_augment = random.choice(images)
                # Apply random transformation
                augmented_image = random_transform(image_to_augment)
                # Add augmented image and label
                new_images.append(augmented_image)
                new_labels.append(labels[0])  # Assume all images in the class have the same label
    
    return new_images, new_labels

Use the *augment_data()* function to augment the training data.

In [ ]:
# Use this cell to augment your training data

# Augment your training data using the "augment_data()" function
X_train, y_train = augment_data(X_train, y_train, 12)

Visualize the class balance in the augmented training data set using a bar plot or a count plot.

In [ ]:
train_df = pd.DataFrame(zip(X_train, y_train), columns=['image', 'label']).groupby(by='label').count()

In [ ]:
# Use this cell to visualize the class balance in the training data set
ax = sns.barplot(train_df.reset_index(), x='label', y='image')
ax.bar_label(ax.containers[0])
ax.set_title("Training data length after augmentation.")
plt.savefig("trainig_data_after_augmentaiton.png")

### Checklist
- Experimented with the *augment_data()* function and made sure that it is working properly
  - For example, if the *augment_data()* function is used on the original training data with a *data_size_factor* of 1, each class in the output data has 285 images, and the total number of images in the new list is 855. If *data_size_factor* was 2 instead, then each class in the output data has 570 images, and the total number images in the new list is 1710.
- Decided on a suitable value for the augmentation factor and augmented the training data

## Task 11 - Create a simple CNN model using the augmented data and analyze its performance


### Description
In this task, you will retrain your simple CNN model multiple times using the augmented data and record its performance in each training instance. You will then analyze its performance by summarizing its performance over the multiple training trials.

But before you do that, you will need to perform all the data prepration steps that you did earlier so that the data is ready to be fed into CNNs.

In [ ]:
def pre_process_image(image_list: typing.List[PIL], new_image_dims: int):
    resized_imgs = resize_images(image_list, new_image_dims)

    rescaled_imgs = np.array([img_to_array(x) / 255 for x in resized_imgs])

    return rescaled_imgs

In [ ]:
def encode_labels(image_labels: typing.List[str]):
    num_classes: int = len(set(image_labels))

    labels = list(set(image_labels))

    label_encoder = LabelEncoder()

    labels_encoded = label_encoder.fit_transform(image_labels)
    labels_encoded = to_categorical(labels_encoded, num_classes=num_classes)
    return labels_encoded

In [ ]:
X_train = pre_process_image(X_train, new_image_dims)
y_train = encode_labels(y_train)

In [ ]:
# X_test = pre_process_image(X_test, new_image_dims)
# y_test = encode_labels(y_test)

In [ ]:
# Use this cell to set the number of trials for each model training instance

# Set the number of trials for each model training instance
num_trials = 10

Now save the number of epochs and the validation split in a variable.

In [ ]:
# Use this cell to set the number of CNN training epochs and the validation split fraction

# Set the number of epochs for CNN training
n_epochs = 10

# Set the validation split fraction
val_split = 0.2

Finally, create train and evaluate your CNN model.

In [ ]:
inputdims = X_train.shape[1], X_train.shape[2], X_train.shape[3]

In [ ]:
# Use this cell to create, train and evaluate your simple CNN model on the data multiple times and store and view the performance results
training_hist = [None] * num_trials
performance_df = pd.DataFrame()

layers_config = ['c_8_3', 'm_2', 'c_12_3', 'm_2', 'f', 'd_24']
learning_rate_value = 0.001

for i in range(0, num_trials):
    results = []
    cnn = create_cnn(layers_config=layers_config, input_dims=inputdims, num_classes=num_classes, learning_rate_value=learning_rate_value)

    print(f"Training iteration {i}")
    cnn.summary()
    print('\n')
    cnn_history = cnn.fit(X_train, y_train, validation_split=val_split, epochs=n_epochs)

    training_hist[i] = pd.DataFrame(cnn_history.history)
    training_hist[i]['epoch'] = cnn_history.epoch

performance_df = reduce(lambda left, right: pd.merge(left, right, how='outer'), training_hist)
performance_df.loc['Mean'] = performance_df.mean()
performance_df.loc['Median'] = performance_df.median()
performance_df.loc['Max'] = performance_df.max() 

In [ ]:
plt.figure(figsize = (14, 4))

sns.lineplot(data = performance_df, x = 'epoch', y = 'accuracy', color = 'red', label = 'Train')
sns.lineplot(data = performance_df, x = 'epoch', y = 'val_accuracy', color = 'blue', label = 'Validation')
plt.xlabel('Iteration')
plt.ylabel('Accuracy')
plt.title('Accuracy Epochs');

You should be able to observe, from the performance data frame, that the basic CNN model that is trained on the augmented data performs better, in general, than the basic CNN model that you trained earlier.

### Checklist
- Converted the augmented input data into arrays
- Scaled the augmented input data
- Encoded the output data as integers
- Performed one-hot encoding on output data
- Retrained the simple CNN model on the augmented data
- Created a data frame to analyze the performance of the model

# Stage 5 - Optimal Model
In this stage, you will train your CNN model on the augmented data set and tune it for network structure and learning rate. You will do this by completing Task 12 - Tune the CNN Model.

## Task 12 - Tune the CNN Model

### Description

In this task, you will train your CNN model on the augmented data set and tune it for network structure and learning rate.

A grid search is not recommended here since we are working with sparse data. Instead, it is recommended to use simple loops and record the performance of the various model specifications. This also ensures that the same validation split and consequently the same validation data is used, so the model performance analyses are not biased, at least within the context of this assignment.

Additionally note that you are required to restrict the complexity of your model. In general, more complex networks have a higher likelihood of overfitting. As part of this assignment, you are required to make sure that your networks contain no more than 500,000 trainable parameters.

Note that you are required to report a performance data frame as you did earlier for each of the model specifications that you consider in this stage. You are free to report these as separate data frames or within a single data frame.

For instance, if you consider 2 values for network configuration and 2 values for learning rate, then there are 4 unique model specifications. Consequently, you must train each of these model types multiple times (as specified earlier) and record the performance in each trial. You may, therefore, report 4 separate data frames (whose format you should be familiar with by this stage) or a single data frame that contains all the necessary information.

Tune your CNN model for network structure and learning rate.

In [ ]:
# Use this cell to tune your CNN model for network structure and learning rate
layers_config_list = [
    # Small network configurations
    ['c_12_3', 'm_2', 'c_14_3', 'm_2', 'f', 'd_24'],
    ['c_16_3', 'm_4', 'c_24_3', 'm_2', 'f', 'd_32'],
]

learning_rate_list = [0.01, 0.001, 0.002]

# Initialize dataframe to record performance metrics
results_df = pd.DataFrame(columns=['Layers Configuration', 'Learning Rate', 'Train Accuracy', 'Validation Accuracy'])

# Define maximum allowable trainable parameters
max_trainable_params = 5_00_000

# Loop through combinations of network configurations and learning rates
for layers_config in layers_config_list:
    for learning_rate in learning_rate_list:
        # Create and compile the model
        cnn = create_cnn(layers_config=layers_config, input_dims=inputdims, learning_rate_value=learning_rate, num_classes=num_classes)
        cnn.summary()

        if cnn.count_params() > max_trainable_params:
            print(f"Max {max_trainable_params} trainable params allowed! Please create a small network.")
            break

        cnn_history = cnn.fit(X_train, y_train, epochs=n_epochs, validation_split=val_split)
        
        
        # Record performance metrics
        train_accuracy = cnn_history.history['accuracy'][-1]
        val_accuracy = cnn_history.history['val_accuracy'][-1]
        
        # Append results to dataframe
        results_df.loc[len(results_df)] = {'Layers Configuration': layers_config,
                                        'Learning Rate': learning_rate,
                                        'Train Accuracy': train_accuracy,
                                        'Validation Accuracy': val_accuracy}
                                       

# Find optimal values for network config and learning rate
optimal_row = results_df.loc[results_df['Validation Accuracy'].idxmax()]
optimal_layer_config = optimal_row['Layers Configuration']
optimal_learning_rate = optimal_row['Learning Rate']

In [ ]:
optimal_row

In [ ]:
print(optimal_layer_config, optimal_learning_rate)

You should take some time and study the results of your hyperparameter tuning. Once you are satisfied with your analysis, decide on the optimal values of the hyperparameters to use.

### Checklist
- Decided on the values of hyperparameters over which to tune the model
- Trained multiple models for each combination of hyperparameters
- The optimal model has good and consistent validation accuracy
- The optimal model doesn't have more than 500,000 trainable parameters

# Stage 6 - Testing
In this stage, you will train your optimal model multiple times on the augmented data set until you are satisfied with its performance on the validation data. You will then test your optimal model on the hold-out test data set, which has not been used up until this point.

## Task 13 - Train your optimal model satisfactorily

### Description

In this section, you will train your optimal model on the augmented data set multiple times until you are satisfied with its performance on the validation data.

In [ ]:
threshold = 0.90
performance_test = (0, 0)

In [ ]:
while performance_test[1] < threshold:
    # Use this cell to train your optimal model on the augment data set until you are satisfied with its validation performance
    optimal_cnn = create_cnn(layers_config=optimal_layer_config, input_dims=inputdims, learning_rate_value=optimal_learning_rate, num_classes=num_classes)
    optimal_cnn.summary()
    optimal_cnn_history = optimal_cnn.fit(X_train, y_train, epochs=10, validation_split=val_split)
    performance_test = optimal_cnn.evaluate(X_test, y_test)
    print(f"****************** The accuracy is {performance_test[1]} ******************")

In [ ]:
# Use this cell to test your optimal model on the hold-out test data set

# Obtain the perfomance metrics of the optimal model on the testing data set using the "evaluate()" method
performance_test = optimal_cnn.evaluate(X_test, y_test)

print('The accuracy of the model on the testing data is {}'.format(performance_test[1]))

In [1]:
tf.keras.models.save_model("best_model")

### Checklist
- Decided on a minimum validation accuracy
- Trained multiple models till that accuracy is achieved
- Your optimal model does well on the testing data

Now that you have completed all the tasks in the assignment, please move on to create your analysis report, and subsequently prepare to submit the required files to the platform.